In [14]:
import pandas as pd
import numpy as np

In [13]:
#u_data-> dataframe으로 변경
u_data = []
with open('./ml-100k/u.data','r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_data.append(line.strip().split('\t'))
    
u_data = pd.DataFrame(u_data, columns=["userID","itemID","rating","timestamp"])  
u_data.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [56]:
#u.item -> dataframe으로 변경
u_item = []
with open("./ml-100k/u.item",'r',encoding='ISO-8859-1') as f:
    for line in f.readlines():
        u_item.append(line.strip().split("|")[:-4]) #README에서 item의 컬럼은 20개 였으나 해당 데이터에는 24개의 컬럼이 존재하여 4개의 컬럼을 제외시켰습니다.
        

        
movies_info = """movie id | movie title | release date | video release date |
IMDb URL | unknown | Action | Adventure | Animation |
Children's | Comedy | Crime | Documentary | Drama | Fantasy |
Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
Thriller | War | Western"""        
        
movies_info = movies_info.split(" | ")    

u_item = pd.DataFrame(u_item,columns=movies_info)
u_item

,movie id,movie title,release date,video release date |\nIMDb URL,unknown,Action,Adventure,Animation |\nChildren's,Comedy,Crime,Documentary,Drama,Fantasy |\nFilm-Noir,Horror,Musical,Mystery,Romance,Sci-Fi |\nThriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Four Rooms (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Get Shorty (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1679,1680,Sliding Doors (1998),01-Jan-1998,,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1680,1681,You So Crazy (1994),01-Jan-1994,,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [55]:
#u.user -> dataframe으로 변경
u_user = []
with open("./ml-100k/u.user",'r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_user.append(line.strip().split("|")) #README에서 item의 컬럼은 20개 였으나 해당 데이터에는 24개의 컬럼이 존재하여 4개의 컬럼을 제외시켰습니다.
        

        
user_info = """user id | age | gender | occupation | zip code"""        
        
user_info = user_info.split(" | ")    

u_user = pd.DataFrame(u_user,columns=user_info)
u_user

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [23]:
# 1.전체이용자에서 가장 시청률이 높은 영화 TOP 100
result = u_data.value_counts(subset="itemID", ascending=False)


result = pd.DataFrame(result)
result

,0
itemID,
50,583
258,509
100,508
181,507
294,485
...,...
1352,1
1636,1
1637,1
